#### Progetto di corso per APPLICAZIONI DELL'INTELLIGENZA ARTIFICIALE (AA 2024-2025)
#### Stud: Marzio Della Bosca


Jupyter Notebook con la funzione di estrarre le feature, tramite catch22 e tsfel, dai dati contenuti nel dataset [Moby Act](https://bmi.hmu.gr/the-mobifall-and-mobiact-datasets-2/)

In [1]:
import tsfel
from tqdm import tqdm
import torch
import warnings
warnings.filterwarnings("ignore")
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
import time
from tslearn.utils import to_time_series_dataset, ts_zeros, to_time_series
from tslearn.barycenters import softdtw_barycenter
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.preprocessing import StandardScaler
from tslearn.metrics import dtw
import os
import torch.nn as nn
import torch.optim as optim
from sklearn.model_selection import train_test_split
from aeon.transformations.collection.feature_based import Catch22
from sklearn.experimental import enable_hist_gradient_boosting
from sklearn.ensemble import HistGradientBoostingClassifier
from sklearn.model_selection import GridSearchCV
import random as rnd

In [ ]:
# Caricamento del dataset
# Struttura:
# acc_x, acc_y, acc_z, gyr_x, gyr_y, gyr_z, classe, ...

# (label-classe di riferimento): 1-Walking, 2-Jogging, 3-StairsDwn, 4-StairsUp, 5-Sitting, 6-Standing
#                                7-Running, 8-Biking, 9-LayingDwn, 10-Static, 11-AerobicDan

# Seleziono solo 1,2,3,4,5,6,7, in quanto classi simili a quelle degli altri dataset presi in considerazione
# MobiAct è campionato a 20 Hz (paper: tramite interpolazione e downsampling viene reso da 100 a 20)

mobyActP = pd.DataFrame(np.load('./MOBY_data/mobiact_full_tags.npy'))
print(mobyActP.shape)

(12421968, 12)


In [3]:
mobyAct_subset = mobyActP.iloc[:, :7]
mobyAct_subset.columns = ['acc_x', 'acc_y', 'acc_z', 'gyr_x', 'gyr_y', 'gyr_z', 'classe']

# Filtra il dataset per mantenere solo le righe con classe <= 7
mobyAct_filtered = mobyAct_subset[mobyAct_subset['classe'] <= 7]

print(mobyAct_filtered.head())
print(f"\nShape del dataset filtrato: {mobyAct_filtered.shape}")

# Stampa i valori unici della colonna con indice 6, per verifica filtraggio classi
valori_unici = mobyAct_filtered.iloc[:, 6].unique()
print("\nValori unici nella colonna [6]:", valori_unici)

      acc_x     acc_y     acc_z     gyr_x     gyr_y     gyr_z  classe
0 -1.307938  9.696413 -1.748079 -0.605367  0.099876  0.039095     1.0
1 -1.255622  9.685399 -1.781121 -0.127976  0.050702  0.029322     1.0
2 -1.206497  9.675057 -1.812147 -0.037263  0.070860  0.036957     1.0
3 -1.156879  9.664611 -1.843484 -0.092241  0.076664  0.039401     1.0
4 -1.075953  9.678261 -1.872745 -0.091019  0.066584  0.056200     1.0

Shape del dataset filtrato: (11246698, 7)

Valori unici nella colonna [6]: [1. 2. 3. 4. 5. 6. 7.]


In [46]:
def windowing(data, window_size):
    data = pd.DataFrame(data)
    X, y = [], []

    for i in range(0, len(data) - window_size, window_size):
        window = data.iloc[i:i+window_size]
        X.append(window.values)
        # Calcola la moda solo sull'ultima colonna
        label_mode = window.iloc[:, -1].mode()[0]  # questo sistema mi serve quando dataset non è diviso in classi
        y.append(label_mode)                   

    return np.array(X), np.array(y)

def collect_by_class(data):
    # Raggruppa per "classe"
    grouped = data.groupby('classe')
    # Crea una lista per classi da 1 a 7 (se la classe manca, inserisce una lista vuota)
    return [grouped.get_group(i).values.tolist() if i in grouped.groups else [] for i in range(1, 8)]

def extract_catch22_features(data):
    # data shape: (n_samples, window, n_features)
    n_samples, window, n_channels = data.shape
    catch22 = Catch22()
    features = np.zeros((n_samples, 22, n_channels)) # prealloco la memoria in ram, velocizzo di molto i calcoli sfruttando 
                                                     # meglio anche le funzioni messe a disposizione da Numpy e Pandas

    for i in tqdm(range(n_samples), desc="Estrazione CATCH22"):
        for ch in range(n_channels):
            series = data[i, :, ch]
            if np.all(series == series[0]):
                series = series + np.random.normal(0, 1e-6, size=series.shape)
            try:
                feats = catch22.fit_transform([series]).flatten()
            except Exception:
                feats = np.full(22, np.nan)
            features[i, :, ch] = feats

    return features

def extract_tsfel_features(data, cfg_kind):
    # Carica la configurazione di default
    cfg = tsfel.get_features_by_domain(cfg_kind) # ovvero informazioni per domini di informazione 'statistical', 'temporal', 'spectral'
    # Calcola il numero di feature con un test rapido (primo esempio e un solo canale) (sempre per ottimizzare come per catch22)
    # data shape: (n_samples, window, n_features)
    n_samples, window, n_channels = data.shape
    test_feats = tsfel.time_series_features_extractor(cfg, pd.DataFrame(data[0, :, 0]), verbose=0)
    n_feat = test_feats.shape[1]
    features_3d = np.zeros((n_samples, n_feat, n_channels))

    for i in tqdm(range(n_samples), desc="Estrazione TSFEL"+ cfg_kind):
        for ch in range(n_channels):  # per ogni canale (6)
            signal = pd.DataFrame(data[i, :, ch])  # un canale come serie temporale
            feats = tsfel.time_series_features_extractor(cfg, signal, verbose=0, n_jobs=-1)
            features_3d[i, :, ch] = feats.values.flatten()
    
    return features_3d

In [ ]:
window_size = 40 # imposto una finestra a 2 secondi
moby_raw_w40, moby_labels_w40 = windowing(mobyAct_filtered, 40)

print(f"Shape Moby raw: {moby_raw_w40.shape}")
print(f"Shape Moby labels: {moby_labels_w40.shape}")

Shape Moby raw: (281167, 40, 7)
Shape Moby labels: (281167,)


In [22]:
moby_raw_w40 = moby_raw_w40[:, :, :-1]
print(f"Shape Moby raw: {moby_raw_w40.shape}")

Shape Moby raw: (281167, 40, 6)


In [27]:
# Estraggo feature da catch22, non scalo ancora i dati in quanto non voglio fare estrazioni sbagliate
startT = time.time()
X_catch22 = extract_catch22_features(moby_raw_w40)
print(f"Tempo di estrazione feature catch22: {time.time() - startT:.2f} secondi")

Estrazione CATCH22: 100%|██████████| 281167/281167 [17:12<00:00, 272.33it/s]

Tempo di estrazione feature catch22: 1032.47 secondi


In [28]:
print(f"Shape X_200_catch22: {X_catch22.shape}")
np.save('mobiact_catch22_40.npy', X_catch22)

Shape X_200_catch22: (281167, 22, 6)


In [47]:
startT = time.time()
X_tsfel_stat = extract_tsfel_features(moby_raw_w40, 'statistical') # non posso fare tutti i domini per questioni di ram necessaria
                                                                   # quindi estraggo solo da dom. statistico e temporale 
print(f"\nTempo di estrazione feature tesfel con statistical: {time.time() - startT:.2f} secondi")
startT = time.time()
X_tsfel_temp = extract_tsfel_features(moby_raw_w40, 'temporal')
print(f"\nTempo di estrazione feature tesfel con temporal: {time.time() - startT:.2f} secondi")

Estrazione TSFELstatistical: 100%|██████████| 281167/281167 [53:59<00:00, 86.80it/s]



Tempo di estrazione feature tesfel con statistical: 3239.13 secondi


Estrazione TSFELtemporal: 100%|██████████| 281167/281167 [27:00<00:00, 173.49it/s]


Tempo di estrazione feature tesfel con temporal: 1620.65 secondi


In [48]:
X_tsfel_concat = np.concatenate((X_tsfel_stat, X_tsfel_temp), axis=1)
print("Shape concatenata:", X_tsfel_concat.shape)

Shape concatenata: (281167, 45, 6)


In [49]:
np.save('mobiact_tsfel_40.npy', X_tsfel_concat)